In [1]:
from flurs.datasets import csv_loader
from flurs.recommender import BRISMFRecommender, MFRecommender
from flurs.forgetting import ForgetUnpopularItems, NoForgetting, MappedUserFactorFading, UserFactorFading, SDUserFactorFading
from flurs.evaluator import Evaluator

import logging
import os
import sys

ML_PATH = 'D:\\recsys\\datasets\\MovieLens\\ml-1m\\ml-1m-gte.csv'
EXP_NAME = "M2"

program = os.path.basename(ML_PATH)
logger = logging.getLogger(program)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info('running %s' % ' '.join(sys.argv))

logging.info('converting data into FluRS input object')
data = csv_loader(ML_PATH)

2019-03-17 01:08:17,319 : INFO : running D:\Anaconda3\envs\recsys\lib\site-packages\ipykernel_launcher.py -f C:\Users\Eduardo Ferreira\AppData\Roaming\jupyter\runtime\kernel-c8c11876-00db-40ae-91cb-23706debe5f0.json
2019-03-17 01:08:17,326 : INFO : converting data into FluRS input object


In [2]:
class Experimenter:
    def __init__(self):
        self._recommenders = []
    def append(self, recommender_list, forgetting_list, parameters_list):
        for Recommender in recommender_list:
            for p in parameters_list:
                for Forgetting in forgetting_list:
                    self._recommenders.append(Recommender(forgetting = Forgetting(alpha=p)))
    def __getitem__(self, position):
        return self._recommenders[position]

In [3]:
experimenter = Experimenter()
recommender_list = []
recommender_list.append(MFRecommender)

experimenter.append(recommender_list, [ForgetUnpopularItems], [1.2])
# experimenter.append(recommender_list, [NoForgetting], [None])
# experimenter.append(recommender_list, [UserFactorFading], [0.99])
# experimenter.append(recommender_list, [SDUserFactorFading], [1.08])
# experimenter.append(recommender_list, [MappedUserFactorFading], [1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6])

In [4]:
experiments = []
for rec in experimenter:
    logging.info('initialize recommendation model {} with {}'.format(rec.__class__.__name__, rec.forgetting.__class__.__name__))
    rec.initialize()
    evaluator = Evaluator(rec, data.can_repeat)

    n_batch_train = int(data.n_sample * 0.2)  # 20% for pre-training to avoid cold-start
    n_batch_test = int(data.n_sample * 0.3)  # 30% for evaluation of pre-training
    batch_tail = n_batch_train + n_batch_test

    # pre-train
    # 20% for batch training | 30% for batch evaluate
    # after the batch training, 30% samples are used for incremental updating
    logging.info('batch pre-training before streaming input')
    evaluator.fit(
        data.samples[:n_batch_train],
        data.samples[n_batch_train:batch_tail],
        n_epoch=1  # single pass even for batch training
    )

    # 50% incremental evaluation and updating
    logging.info('incrementally predict, evaluate and update the recommender')
    experiments.append((evaluator.evaluate(data.samples[batch_tail:]), rec.__class__.__name__, rec.forgetting.__class__.__name__, rec.forgetting.alpha))

2019-03-17 01:08:17,856 : INFO : initialize recommendation model MFRecommender with ForgetUnpopularItems
2019-03-17 01:08:17,857 : INFO : batch pre-training before streaming input
2019-03-17 01:09:15,915 : INFO : incrementally predict, evaluate and update the recommender


In [ ]:
for evaluator, recommender, forgetting, param in experiments:
    filename = "D:\\recsys\\flurs\\results\\{}_{}_{}_{}.dat".format(recommender, forgetting, param, EXP_NAME)
    logging.info("Abrindo arquivo {} ...".format(filename))
    with open(filename, 'w+') as f:
        logging.info("Começando a gerar resultados ...")
        for instance in evaluator:
            f.write(str(instance))
    logging.info("Arquivo {} completo.".format(filename))
    

2019-03-17 01:09:15,932 : INFO : Abrindo arquivo D:\recsys\flurs\results\MFRecommender_ForgetUnpopularItems_1.2_M2.dat ...
2019-03-17 01:09:15,936 : INFO : Começando a gerar resultados ...
